In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from numpy.linalg import norm
from collections import Counter, defaultdict
from scipy.sparse import csr_matrix
import nltk
import string
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
import re
from nltk.corpus import stopwords
from string import digits
#nltk.download("wordnet")
from nltk.stem import PorterStemmer, WordNetLemmatizer
from collections import Counter
from scipy.sparse import csr_matrix
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report


In [18]:
# open docs file and read its lines
with open("train.dat", "r") as fh:
    lines = fh.readlines() 
    
train_arr = []
train_lbl = []

for i in range(len(lines)):
    train_arr.append(lines[i].lower()[2:])
    train_lbl.append(lines[i][0:1])

print train_arr[0:2]
print train_lbl[0:2]

['catheterization laboratory events and hospital outcome with direct angioplasty for acute myocardial infarction to assess the safety of direct infarct angioplasty without antecedent thrombolytic therapy, catheterization laboratory and hospital events were assessed in consecutively treated patients with infarctions involving the left anterior descending (n = 100 patients), right (n = 100), and circumflex (n = 50) coronary arteries. the groups of patients were similar for age (left anterior descending coronary artery, 59 years; right coronary artery, 58 years; circumflex coronary artery, 62 years), patients with multivessel disease (left anterior descending coronary artery, 55%; right coronary artery, 55%; circumflex coronary artery, 64%), and patients with initial grade 0/1 antegrade flow (left anterior descending coronary artery, 79%; right coronary artery, 84%; circumflex coronary artery, 90%). cardiogenic shock was present in eight patients with infarction of the left anterior desce

In [19]:
#remove short words
def filterLen(tdocs, minlen):
    return [ ' '.join(t for t in d if len(t) >= minlen ) for d in tdocs ]

trainDocs = [l.split() for l in train_arr]
train_filtered = filterLen(trainDocs,4)

#lemmatize and remove stop words
def preProcess(data_arr):
    lemmatiser = WordNetLemmatizer()
    pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
    #print train_arr[0:10]
    for i in range(len(train_arr)):
        data_arr[i] = pattern.sub('', data_arr[i])
        data_arr[i] = data_arr[i].translate(None, digits)
        #data_arr[i] = nltk.word_tokenize(lemmatiser.lemmatize(data_arr[i],pos="v").translate(None, string.punctuation))
    return data_arr

train_arr = preProcess(train_filtered)
print train_arr[0:2]

['catheterization laboratory events hospital outcome direct angioplasty acute myocardial infarction assess safety direct infarct angioplasty without antecedent thrombolytic therapy, catheterization laboratory hospital events assessed consecutively treated patients infarctions involving left anterior descending patients), right ), circumflex coronary arteries. groups patients similar (left anterior descending coronary artery, years; right coronary artery, years; circumflex coronary artery, years), patients multivessel disease (left anterior descending coronary artery, %; right coronary artery, %; circumflex coronary artery, %), patients initial grade antegrade flow (left anterior descending coronary artery, %; right coronary artery, %; circumflex coronary artery, %). cardiogenic shock present eight patients infarction left anterior descending coronary artery, four infarction right coronary artery, four infarction circumflex coronary artery. major catheterization laboratory events (cardi

In [20]:
#create TDIF matrix
vectorizer = CountVectorizer()
tfidf = TfidfTransformer()
vectorizer.fit(train_arr)
x_train_arr = vectorizer.transform(train_arr)
tfidf.fit(x_train_arr)
x_train = tfidf.transform(x_train_arr)
print x_train.shape


(14438, 34727)


In [21]:
# open docs file and read its lines - for test mode
# with open("test.dat", "r") as fh:
#     linesTest = fh.readlines() 
    
# for i in range(len(linesTest)):
#     test_arr.append(linesTest[i].lower())
  
# test_arr = preProcess(test_arr)

# vectorizer.fit(test_arr)
# x_test_arr = vectorizer.transform(test_arr)
# tfidf.fit(x_test_arr)
# x_test = tfidf.transform(x_test_arr)

# print x_train.shape
# print len(train_lbl)
# print x_test.shape


In [28]:
#split to train and test
train_lbl = np.asarray(train_lbl)
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(x_train, train_lbl, test_size=0.25, random_state=42)
print X_train.shape
print X_test.shape

print Y_train.shape
print Y_test.shape


(10828, 34727)
(3610, 34727)
(10828,)
(3610,)


In [58]:
# from sklearn.feature_selection import SelectKBest,mutual_info_classif,f_classif,VarianceThreshold,chi2,SelectPercentile,SelectFdr
# featureSelector1 = SelectKBest(f_classif, k=200)
# X_new = featureSelector1.fit_transform(X_train,Y_train)
# print X_new.shape

# X_test_new = featureSelector1.transform(X_test)
# print X_test_new.shape

X_new = X_train
X_test_new = X_test

In [65]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression, Perceptron
from sklearn.svm import LinearSVC
#from sklearn.neural_network import MLPClassifier
kfold = model_selection.StratifiedKFold(n_splits=5)
#model = ExtraTreesClassifier(random_state=42, class_weight ='balanced_subsample', n_estimators=200,min_samples_split = 5)
model = SGDClassifier(loss='hinge', penalty='l2',alpha=0.0004, max_iter=100, random_state=42)
#model = LogisticRegression(class_weight='balanced', C=0.1)
#model = MLPClassifier(max_iter=18)
#model = AdaBoostClassifier(LinearSVC(penalty='l1', dual=False,tol=1e-3, class_weight='balanced',C=0.1),algorithm='SAMME')

model.fit(X_new,Y_train)
predict = model.predict(X_test_new)
#Cross Validate
#results = model_selection.cross_val_score(model, x_train, train_lbl, cv=kfold, scoring='f1_micro')
#print(results.mean())

print '\n clasification report:\n', classification_report(Y_test,predict)
